<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone pinecone-notebooks openparse[ml] html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━

## Configuração de Variaveis

In [2]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

## Pré-processamento de dados

### Download do dataset

In [3]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

--2024-08-29 23:28:40--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1383621 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.32M   850KB/s    in 1.6s    

2024-08-29 23:28:42 (850 KB/s) - ‘data/ppc_2018.pdf’ saved [1383621/1383621]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/My\ Drive/tcc/PPC_2024.pdf data/ppc_2024.pdf



### Transformação PDF -> HTML -> Markdown




In [ ]:
import openparse
import html2text
from langchain_core.documents import Document


def parse_document_to_documents(file_path, source_name):
    # Parse the document using OpenParse
    parser = openparse.DocumentParser(table_args={"parsing_algorithm": "pymupdf", "table_output_format": "markdown"})
    document = parser.parse(file_path)

    documents = []

    # Iterate over the nodes and convert each to a Document
    for node in document.nodes:
        html_content = node.text

        if html_content:
            document_chunk = Document(
                metadata={'source': source_name},
                page_content=html_content
            )
            documents.append(document_chunk)

    return documents


In [ ]:
ppc_2018_chunks = parse_document_to_documents('data/ppc_2018.pdf', 'ppc2018')

In [ ]:
ppc_2024_chunks = parse_document_to_documents('data/ppc_2024.pdf', 'ppc2024')

### Geração de embeddings e indíces do banco vetorial

In [ ]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [ ]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [ ]:
import time

index_name = "index-ppc-markdown-dev"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [ ]:
!pip install -qU langchain-huggingface langchain langchain-community unstructured==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents(file_path):
    """Load documents from a Markdown file using UnstructuredMarkdownLoader."""
    # Usar um loader específico para Markdown
    loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()
    return documents

def split_documents_into_chunks(documents, chunk_size=2000, chunk_overlap=400):
    """Split the loaded documents into text chunks based on Markdown structure."""
    # Usando RecursiveCharacterTextSplitter para respeitar a estrutura do Markdown
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n# ", "\n## ", "\n### ", "\n", " "]
    )
    texts = text_splitter.split_documents(documents)
    return texts

def create_documents_from_chunks(texts, source_name):
    """Create Document objects from text chunks."""
    documents = [
        Document(page_content=text.page_content, metadata={"source": source_name}) for text in texts
    ]
    return documents

def generate_unique_ids(num_ids):
    """Generate a list of unique UUIDs."""
    return [str(uuid4()) for _ in range(num_ids)]

def add_documents_to_vector_store(vector_store, documents, uuids):
    """Add documents with unique IDs to the vector store."""
    print(vector_store.add_documents(documents=documents, ids=uuids))


### Ingestão PPC 2018

In [ ]:
# file_path = "data/ppc_2018.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2018.md")


In [ ]:
uuids = generate_unique_ids(len(ppc_2018_chunks))
add_documents_to_vector_store(vector_store, ppc_2018_chunks, uuids)

['a50d3435-ebee-456f-9543-3421a4ebb2fe', 'c5c29264-9251-486a-bbee-1615595d8a04', '1f9cc19c-17f9-4b73-96cc-d64c31fbd175', '0dab9f53-a830-4c7d-a086-9a6c6296fc28', '30956b4c-5811-47e8-8ac0-979403c911e9', 'edcca4ed-de15-44ea-a7a4-00ade11c8603', '00147062-e83e-433e-b5e2-8d2e30c11aa6', '0715f5dc-c81c-4d90-96a8-dc770fc1dce4', 'aab5caad-746d-4e87-a1d5-6288106159a4', '41a25be7-d85f-4889-a107-63c795b9fa11', 'b7f5f55f-26f2-442a-9c61-a547f7b55976', 'b246afb3-50d2-42f7-a8f5-75904259ad55', 'e0f19a8d-7cba-4ee8-bb58-f57083661426', '227ee6b8-426f-4a84-b04d-8ed30b003d0c', 'c033e10e-2c09-4874-8f76-361466c9903e', '0ed657b3-0b36-4509-8408-ab9ea15e87a7', '484eda7a-e3b0-4063-ade8-f2a50014cef5', 'b7971593-5fa0-47c1-940d-df5dc5865d66', 'f88f9e72-c04f-410d-8822-06fbcc024e79', 'dc396e37-999f-4bcb-882a-02d701c59494', '1485f263-7ae3-449c-b0ea-01d4547d1c96', '57c86807-ad44-4cf1-9c37-168d8bd44bad', '9c6841bf-3f32-4f3a-8735-ab9e4bd01c76', 'ac80a4d3-32bb-4232-8846-9905512a6196', 'e8a703f4-52f9-43fd-ba7a-54bc2cbb26a0',

### Ingestão PPC 2024

In [ ]:
# file_path = "data/ppc_2024.md"
# text = load_documents(file_path)
# chunks = split_documents_into_chunks(text)
# documents = create_documents_from_chunks(chunks, source_name="ppc2024.md")


In [ ]:
uuids = generate_unique_ids(len(ppc_2024_chunks))
add_documents_to_vector_store(vector_store, ppc_2024_chunks, uuids)

['e5d296ca-fd09-4870-978f-2b08ff183299', '3dc4f2ac-f546-47a5-a8b5-17d0e526e5d4', '6ccca382-52f0-4728-ae6a-60ecd6365a0c', '7c7615ef-37cc-471f-bddf-a73e58b08f71', '1f30205b-6fb8-4162-b7f3-b226cd734335', 'dfec970f-3c36-4e54-9082-d110cda4896e', '1f267775-4bfc-4994-8f24-4afec2ecb5db', 'f160dab6-765a-4da7-88ca-f11925744503', '4f558dbc-8a81-4f1d-8262-f570a4bc09af', '3b2b2ca1-dcbf-460e-b2f8-3c5396576f49', '65514912-0b23-457c-88d0-bed61a89c60e', 'cf6951d1-c39f-445f-bdff-27470061e0c5', '171c3535-e401-408f-97fb-930aaf7331a1', 'f437ddcd-f1cf-4d59-9152-b67759852cfe', '9fcb43ea-d712-46ca-87a1-7c0fa47f8ff3', 'a155ef54-57e0-4dc3-8f48-7402015e833d', 'fd3f8536-1265-446a-aa50-35f2f42f9341', 'a268736e-8372-483a-af2c-88bab9f356b0', 'f6979ad3-f9d5-4e83-b908-74a21926e4e9', '62e7335f-70cd-4b67-b45f-f89aca2e5216', 'a1bea5d8-3dd7-4dc0-9774-4578182eca80', 'c91d737d-16ce-4ae1-aa58-a465c31eee49', 'cbdaf028-2b50-4618-85f2-3ba078672d3d', '953451ee-d6b2-4eb3-ab5c-6d8424a1f6d8', '182614eb-6ad5-41e5-ad4f-96306103e598',

## Geração da resposta

In [ ]:
!pip install -U langchain-groq

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)



def query_expansion(query):

    query_expansion_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""Esse é minha pergunta: {query}
    Me retorne uma string com 5 jeitos de escrever essa mesma pergunta em portugues.
    Considere um contexto academico de uma universidade.
    Retorne apenas as strings separadas com virgula sem nenhuma frase introdutória.
    """

    # Criando o prompt para a consulta expandida
    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    # Gerando a lista de consultas expandidas
    chain = prompt_template | query_expansion_llm
    response = chain.invoke({})
    return response.content


def get_context(query, source_name = 'ppc2024.md'):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    return  '\n'.join([result.page_content for result in results])




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua tarefa é fornecer respostas precisas sobre qualquer questão relacionada ao PPC, utilizando o contexto fornecido para guiar sua resposta."
                "Utilize apenas o contexto informado como referencia"
                "Caso a resposta não está no contexto, informe que não sabe a resposta e recomende consultar o documento original."

                "Responda exclusivamente em portugues brasileiro."
            ),
            (
            "human",
            f"Dado o contexto: {contexto}\n\nPergunta: {pergunta}. Utilize apenas o contexto fornecido para responder"),
        ]
    )
    return prompt

def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    return response


def get_answer(query, source_name):
    expanded_query = query_expansion(query)
    print(expanded_query)
    context = get_context('Atividades de Ensino', source_name)
    prompt = build_prompt(context, query)
    response = get_completion(prompt)

    return context, response.content


In [ ]:
query = "Qual é o limite de horas para Atividades de Ensino?"
context, answer = get_answer(query, "ppc2024")
answer

Qual o limite máximo de horas para atividades de ensino? ,  
Qual a carga horária máxima permitida para atividades de ensino? ,
Até quantas horas por semana um professor pode dedicar a atividades de ensino? ,
Existe um limite de horas para atividades de ensino na universidade? ,
Em que medida a carga horária de atividades de ensino é regulamentada? 



'O limite de horas para Atividades de Ensino é de 120 horas. Essa informação é encontrada no contexto fornecido, especificamente no quadro que apresenta as "Atividades Curriculares Complementares (ACCs)". A categoria "Atividades de Ensino" tem um limite de horas de 120, o que responde à pergunta.'

In [ ]:
context

'participação ativa na aplicação e no acompanhamento do Projeto Pedagógico. <br><br>O Projeto Pedagógico do Curso prevê conteúdos, práticas, atividades de extensão e de <br><br>895  pesquisa implementados por meio de componentes curriculares obrigatórios e optativos, <br><br>sendo ministrados em salas de aulas e em laboratórios. O discente deve colocar em prática os <br><br>conteúdos através de projetos relacionados aos componentes, atividades curriculares <br><br>complementares, atividades de extensão e trabalho de conclusão de curso. <br><br>A organização curricular obedecerá aos princípios pedagógicos, metodológicos e <br><br>900  epistemológicos da proposta curricular da UFFS, tendo como subsídio as orientações <br><br>apresentadas no Projeto Pedagógico Institucional (PPI). \n1255  *8.7.2 Atividades curriculares complementares (Normatização no ANEXO I)*<br><br>As Atividades Curriculares Complementares (ACCs) constituem ações que visam a <br><br>complementação do processo ensino-apr

In [ ]:
results = vector_store.similarity_search(
    "Limite de Atividades de ensino",
    k=2,
    filter={"source": "ppc2024"},
)
results

[Document(metadata={'source': 'ppc2024'}, page_content='<table border="1">\n<tr><th>II</th><th>Atividades de Ensino (Limite horas: 120)</th><th></th><th></th><th></th></tr>\n<tr><td>II</td><td>Atividades de Ensino (Limite horas: 120)</td><td> </td><td> </td><td> </td></tr>\n<tr><td>Categoria</td><td>Descrição da Atividade</td><td>Limite de\nHoras</td><td>Horas por\nEvento</td><td>Documento\ncomprobatório</td></tr>\n<tr><td>2.1</td><td>Programas de monitoria</td><td>100\nDRAFT</td><td>total de horas\nde participação</td><td>Certificado, ou declaração\ndo responsável pelo\nprograma.</td></tr>\n<tr><td>2.2</td><td>Participação em grupos de\nestudos formais do curso de\nCiência da Computação da\nUFFS</td><td>60</td><td>total de horas\nde participação</td><td>Certificado, ou declaração\ndo docente responsável\npelo grupo.</td></tr>\n<tr><td>III</td><td>Outras Atividades Complementares (Limite horas: 120)</td><td> </td><td> </td><td> </td></tr>\n<tr><td>Categoria</td><td>Descrição da Ativida